In [11]:
import pandas as pd

df_monthly = pd.read_csv('/Users/vietanh/Documents/doan/preprocess/monthly_data.csv')
df_monthly['Date'] = pd.to_datetime(df_monthly['Date'])
df_monthly['year-month'] = df_monthly['Date'].dt.to_period('M').astype(str)

print(df_monthly.head())


  Ticker       Date   Open   High    Low  Close  Volume       VWAP year-month
0    A32 2018-10-31  12.85  12.85  12.85  12.85       0   0.000000    2018-10
1    A32 2018-11-30  12.85  15.88  12.85  15.88    1000  14.870000    2018-11
2    A32 2018-12-31  15.88  15.88  13.90  15.84    2200  15.101458    2018-12
3    A32 2019-01-31  16.78  16.78  16.78  17.60     200  15.216275    2019-01
4    A32 2019-02-28  16.78  16.78  12.10  12.10     300  15.090090    2019-02


In [ ]:
df_monthly = pd.read_csv('/Users/vietanh/Documents/doan/preprocess/monthly_data.csv')

df_monthly['Date'] = pd.to_datetime(df_monthly['Date'])
df_monthly['year-month'] = df_monthly['Date'].dt.to_period('M').astype(str)

monthly_grouped = df_monthly.groupby(['Ticker', 'year-month']).agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).reset_index()

monthly_grouped['AdjClose'] = monthly_grouped['Close']

monthly_grouped = monthly_grouped.sort_values(by=['Ticker', 'year-month'])

monthly_grouped['ret'] = monthly_grouped.groupby('Ticker')['Close'].pct_change()
monthly_grouped['Volatility'] = monthly_grouped.groupby('Ticker')['ret'].rolling(3).std().reset_index(level=0, drop=True)
monthly_grouped['HL'] = monthly_grouped['High'] - monthly_grouped['Low']
monthly_grouped['LO'] = monthly_grouped['Close'] - monthly_grouped['Low']
monthly_grouped['SR'] = monthly_grouped['Close'] - monthly_grouped['Open']
monthly_grouped['PM'] = (monthly_grouped['High'] + monthly_grouped['Low']) / 2
monthly_grouped['MDD'] = monthly_grouped.groupby('Ticker')['Close'].apply(lambda x: (x / x.cummax()) - 1).reset_index(level=0, drop=True)
monthly_grouped['TVV'] = monthly_grouped.groupby('Ticker')['Volume'].rolling(3).std().reset_index(level=0, drop=True)

prev_close = monthly_grouped.groupby('Ticker')['Close'].shift(1)
tr = pd.concat([
    monthly_grouped['High'] - monthly_grouped['Low'],
    (monthly_grouped['High'] - prev_close).abs(),
    (monthly_grouped['Low'] - prev_close).abs()
], axis=1).max(axis=1)
monthly_grouped['ATR'] = tr.groupby(monthly_grouped['Ticker']).rolling(3).mean().reset_index(level=0, drop=True)

monthly_grouped['Median_HL'] = (monthly_grouped['High'] + monthly_grouped['Low']) / 2


monthly_grouped['fore_di_rt'] = monthly_grouped.groupby('Ticker')['Close'].shift(-1) / monthly_grouped['Close'] - 1
monthly_grouped['variation_t'] = monthly_grouped.groupby('Ticker')['Close'].pct_change()


monthly_grouped['ma7_t'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: x.rolling(7).mean())
monthly_grouped['ma14_t'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: x.rolling(14).mean())
monthly_grouped['ma21_t'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: x.rolling(21).mean())
monthly_grouped['s_d7_t'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: x.rolling(7).std())

monthly_grouped['rf'] = 0.005  # fixed monthly rf
monthly_grouped['RET'] = monthly_grouped['ret'] - monthly_grouped['rf']

for col in ['SK','KU','RSI','rsq','mvol','mlo','rvni','rsqvni','mlovni','mvolvni','Mkt']:
    monthly_grouped[col] = np.nan

required_columns = [
    'year-month','Ticker','Open','High','Low','Close','AdjClose','Volume',
    'ret','Volatility','HL','LO','SR','PM','MDD','TVV','ATR','SK','KU','RSI',
    'Median_HL','fore_di_rt','variation_t','ma7_t','ma14_t','ma21_t','s_d7_t',
    'rf','RET','rsq','mvol','mlo','rvni','rsqvni','mlovni','mvolvni','Mkt'
]
monthly_final = monthly_grouped[required_columns]

monthly_result_path = "/Users/vietanh/Documents/doan/preprocess/monthly_data_final.csv"
monthly_final.to_csv(monthly_result_path, index=False)




In [13]:
monthly_final['year-month'] = pd.to_datetime(monthly_final['year-month'])

start_month = pd.to_datetime("2013-01")
end_month = pd.to_datetime("2025-03")

filtered_df = monthly_final[
    (monthly_final['year-month'] >= start_month) &
    (monthly_final['year-month'] <= end_month)
]

tickers_start = filtered_df[filtered_df['year-month'] == start_month]['Ticker'].unique()
tickers_end = filtered_df[filtered_df['year-month'] == end_month]['Ticker'].unique()
tickers_valid = set(tickers_start).intersection(set(tickers_end))

final_filtered_df = filtered_df[filtered_df['Ticker'].isin(tickers_valid)]

filtered_output_path = "/Users/vietanh/Documents/doan/preprocess/monthly_data_input.csv"
final_filtered_df.to_csv(filtered_output_path, index=False)




In [ ]:
import pandas as pd

file_path = "/Users/vietanh/Documents/doan/preprocess/bubble.csv"
df = pd.read_csv(file_path)

unique_tickers = df['Ticker'].nunique()

tickers_list = df['Ticker'].unique()

print(f"Số lượng mã cổ phiếu: {unique_tickers}")
print("Danh sách các mã:", tickers_list)


Số lượng mã cổ phiếu: 721
Danh sách các mã: ['AAA' 'AAM' 'ABI' 'ABT' 'ACB' 'ACC' 'ACE' 'ACL' 'ADC' 'ADP' 'AGF' 'AGM'
 'AGR' 'ALT' 'ALV' 'AMC' 'AME' 'AMV' 'ANV' 'APC' 'APG' 'API' 'APP' 'APS'
 'ARM' 'ASA' 'ASM' 'ASP' 'ATA' 'AVF' 'BBC' 'BBS' 'BCC' 'BCE' 'BDB' 'BED'
 'BHC' 'BHP' 'BIC' 'BKC' 'BLF' 'BMC' 'BMI' 'BMJ' 'BMP' 'BPC' 'BRC' 'BSC'
 'BSI' 'BST' 'BT6' 'BTG' 'BTH' 'BTP' 'BTS' 'BTT' 'BTW' 'BVG' 'BVH' 'BVN'
 'BVS' 'BWA' 'BXH' 'C21' 'C32' 'C47' 'C92' 'CAD' 'CAN' 'CAP' 'CCI' 'CCL'
 'CCM' 'CDC' 'CHP' 'CI5' 'CID' 'CIG' 'CII' 'CJC' 'CKV' 'CLC' 'CLG' 'CLW'
 'CMC' 'CMG' 'CMI' 'CMS' 'CMT' 'CMV' 'CMX' 'CNG' 'CNT' 'COM' 'CPC' 'CSC'
 'CSM' 'CT3' 'CT6' 'CTA' 'CTB' 'CTD' 'CTG' 'CTI' 'CTN' 'CTS' 'CTX' 'CVN'
 'CVT' 'CX8' 'CYC' 'D11' 'D2D' 'DAC' 'DAD' 'DAE' 'DAG' 'DBC' 'DBM' 'DBT'
 'DC2' 'DC4' 'DCL' 'DCS' 'DCT' 'DDM' 'DDN' 'DGT' 'DHA' 'DHC' 'DHG' 'DHM'
 'DHT' 'DIC' 'DID' 'DIG' 'DIH' 'DL1' 'DLD' 'DLG' 'DLR' 'DMC' 'DNC' 'DNL'
 'DNM' 'DNP' 'DNT' 'DPC' 'DPM' 'DPP' 'DPR' 'DQC' 'DRC' 'DRH' 'DRL' 'DSN'
 'DST' 

In [ ]:
from tqdm import tqdm
from statsmodels.tsa.stattools import adfuller

monthly_df = pd.read_csv('/Users/vietanh/Documents/doan/preprocess/monthly_data_input.csv')
monthly_df['year-month'] = pd.to_datetime(monthly_df['year-month'])

tickers = monthly_df['Ticker'].unique()
results = []
df_all_bubbles = pd.DataFrame()

for ticker in tqdm(tickers):
    df_ticker = monthly_df[monthly_df['Ticker'] == ticker].copy()

    df_ticker = df_ticker[df_ticker['AdjClose'] > 0]
    if len(df_ticker) < 25:
        results.append({'Ticker': ticker, 'bubble_percent': np.nan})
        continue

    df_ticker['log_price'] = np.log(df_ticker['AdjClose'])
    df_ticker['is_bubble'] = 0

    total_periods = len(df_ticker)
    min_window = 20

    for start in range(0, total_periods - min_window):
        for end in range(start + min_window, total_periods):
            window_data = df_ticker['log_price'].iloc[start:end]

            if window_data.isna().any() or not np.isfinite(window_data).all():
                continue

            try:
                adf_stat, p_value, *_ = adfuller(window_data, maxlag=1, regression='c', autolag=None)
                if p_value < 0.05 and adf_stat < -2.9:
                    df_ticker.loc[df_ticker.index[end], 'is_bubble'] = 1
            except:
                continue

    bubble_percent = round(df_ticker['is_bubble'].sum() / len(df_ticker) * 100, 2)
    results.append({'Ticker': ticker, 'bubble_percent': bubble_percent})

    df_all_bubbles = pd.concat([df_all_bubbles, df_ticker], ignore_index=True)

labeled_path = "/Users/vietanh/Documents/doan/preprocess/bubble.csv"
summary_path = "/Users/vietanh/Documents/doan/preprocess/bubble_summary_percent.csv"
df_all_bubbles.to_csv(labeled_path, index=False)
pd.DataFrame(results).to_csv(summary_path, index=False)



  0%|          | 0/721 [00:00<?, ?it/s]

100%|██████████| 721/721 [28:13<00:00,  2.35s/it]   


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

df = pd.read_csv("/Users/vietanh/Documents/doan/preprocess/bubble.csv")

excluded_cols = ['year-month', 'Ticker', 'is_bubble']

feature_cols = [col for col in df.columns if col not in excluded_cols]

na_ratio = df[feature_cols].isna().mean()

safe_cols = na_ratio[na_ratio == 0].index.tolist()

X = df[safe_cols].copy()
X = X.replace([np.inf, -np.inf], np.nan).dropna()  # loại inf
y = df.loc[X.index, 'is_bubble']  # lấy y tương ứng với X

X = sm.add_constant(X)

ols_model = sm.OLS(y, X).fit()

ols_model.summary().as_text()


'                            OLS Regression Results                            \n==============================================================================\nDep. Variable:              is_bubble   R-squared:                       0.027\nModel:                            OLS   Adj. R-squared:                  0.027\nMethod:                 Least Squares   F-statistic:                     368.6\nDate:                Mon, 21 Apr 2025   Prob (F-statistic):               0.00\nTime:                        23:51:34   Log-Likelihood:                -74885.\nNo. Observations:              105275   AIC:                         1.498e+05\nDf Residuals:                  105266   BIC:                         1.499e+05\nDf Model:                           8                                         \nCovariance Type:            nonrobust                                         \n==============================================================================\n                 coef    std err       

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

file_path = "/Users/vietanh/Documents/doan/preprocess/bubble.csv"
df = pd.read_csv(file_path)

significant_vars = [
    'ret', 'Volatility', 'HL', 'LO', 'SR', 'PM', 'MDD', 'TVV', 'ATR', 'SK',
    'Median_HL', 'fore_di_rt', 'variation_t', 'ma7_t', 'ma14_t', 'ma21_t',
    's_d7_t', 'log_price', 'RET'
]

available_vars = [var for var in significant_vars if var in df.columns]

logit_results = []

sample_tickers = df['Ticker'].unique()[:20]

for ticker in sample_tickers:
    df_ticker = df[df['Ticker'] == ticker].copy()

    df_ticker['is_bubble'] = df_ticker['is_bubble'].shift(-1)

    df_ticker = df_ticker.replace([np.inf, -np.inf], np.nan)
    df_ticker = df_ticker.dropna(subset=['is_bubble'])

    if len(df_ticker) < 20:
        continue

    X = df_ticker[available_vars]
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.fillna(X.mean())
    X = X.fillna(0)
    y = df_ticker['is_bubble']

    # Xử lý Singular matrix
    nunique = X.nunique()
    constant_cols = nunique[nunique <= 1].index.tolist()
    if constant_cols:
        X = X.drop(columns=constant_cols)

    X = X.loc[:, ~X.T.duplicated()]

    if X.shape[1] == 0:
        continue

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    X_train = sm.add_constant(X_train)
    X_test = sm.add_constant(X_test)

    try:
        logit_model = sm.Logit(y_train, X_train).fit(disp=False)
        y_pred_prob = logit_model.predict(X_test)
        y_pred = (y_pred_prob > 0.5).astype(int)

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logit_results.append({
            'Ticker': ticker,
            'Accuracy (%)': round(acc * 100, 2),
            'F1-score': round(f1, 4),
            'Error': np.nan
        })

    except Exception as e:
        logit_results.append({
            'Ticker': ticker,
            'Accuracy (%)': 'Error',
            'F1-score': '-',
            'Error': str(e)
        })

logit_df = pd.DataFrame(logit_results)

logit_df.to_csv("/Users/vietanh/Documents/doan/logit_results_summary_shift.csv", index=False)


logit_df_success = logit_df[logit_df['Accuracy (%)'] != 'Error'].copy()
logit_df_success['Accuracy (%)'] = logit_df_success['Accuracy (%)'].astype(float)
logit_df_success = logit_df_success.sort_values(by='Accuracy (%)', ascending=False)

logit_df_success.to_csv("/Users/vietanh/Documents/doan/logit_results_success.csv", index=False)
logit_df_error = logit_df[logit_df['Accuracy (%)'] == 'Error']
logit_df_error.to_csv("/Users/vietanh/Documents/doan/logit_results_error.csv", index=False)

top5 = logit_df_success.head(5)

print("\n=== Top 5 tickers by Accuracy ===")
print(top5[['Ticker', 'Accuracy (%)', 'F1-score', 'Error']].to_string(index=False))


/opt/anaconda3/envs/code/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/opt/anaconda3/envs/code/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
/opt/anaconda3/envs/code/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/opt/anaconda3/envs/code/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
/opt/anaconda3/envs/code/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/opt/anaconda3/envs/code/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeW


=== Top 5 tickers by Accuracy ===
Ticker  Accuracy (%) F1-score Error
   AMC         90.00   0.9333   NaN
   ABI         86.67   0.6667   NaN
   ACC         86.67   0.7143   NaN
   ADP         83.33   0.7619   NaN
   AGF         83.33      0.8   NaN


/opt/anaconda3/envs/code/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/opt/anaconda3/envs/code/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
